In [1]:
!pip install pandas nltk pyodbc sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.7/334.7 kB 2.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [3]:
customer_reviews_df = pd.read_csv('Sentiment_AnalysisData.csv')
customer_reviews_df.head()

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,23-12-2023,3,"Average experience, nothing special."
1,2,80,19,25-12-2024,5,The quality is top-notch.
2,3,50,13,26-01-2025,4,Five stars for the quick delivery.
3,4,78,15,21-04-2025,3,"Good quality, but could be cheaper."
4,5,64,2,16-07-2023,3,"Average experience, nothing special."


In [4]:
# Initialize the VADER sentiment intensity analyzer for analyzing the sentiment of text data
sia = SentimentIntensityAnalyzer()

In [5]:
# Define a function to calculate sentiment scores using VADER
def calculate_sentiment(review):
    # Get the sentiment scores for the review text
    sentiment = sia.polarity_scores(review)
    # Return the compound score, which is a normalized score between -1 (most negative) and 1 (most positive)
    return sentiment['compound']

# Define a function to categorize sentiment using both the sentiment score and the review rating
def categorize_sentiment(score, rating):
    # Use both the text sentiment score and the numerical rating to determine sentiment category
    if score > 0.05:  # Positive sentiment score
        if rating >= 4:
            return 'Positive'  # High rating and positive sentiment
        elif rating == 3:
            return 'Mixed Positive'  # Neutral rating but positive sentiment
        else:
            return 'Mixed Negative'  # Low rating but positive sentiment
    elif score < -0.05:  # Negative sentiment score
        if rating <= 2:
            return 'Negative'  # Low rating and negative sentiment
        elif rating == 3:
            return 'Mixed Negative'  # Neutral rating but negative sentiment
        else:
            return 'Mixed Positive'  # High rating but negative sentiment
    else:  # Neutral sentiment score
        if rating >= 4:
            return 'Positive'  # High rating with neutral sentiment
        elif rating <= 2:
            return 'Negative'  # Low rating with neutral sentiment
        else:
            return 'Neutral'  # Neutral rating and neutral sentiment

# Define a function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  # Strongly positive sentiment
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  # Mildly positive sentiment
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  # Mildly negative sentiment
    else:
        return '-1.0 to -0.5'  # Strongly negative sentiment

# Apply sentiment analysis to calculate sentiment scores for each review
customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

# Apply sentiment categorization using both text and rating
customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

# Apply sentiment bucketing to categorize scores into defined ranges
customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

# Display the first few rows of the DataFrame with sentiment scores, categories, and buckets
print(customer_reviews_df.head())

   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  23-12-2023       3   
1         2          80         19  25-12-2024       5   
2         3          50         13  26-01-2025       4   
3         4          78         15  21-04-2025       3   
4         5          64          2  16-07-2023       3   

                             ReviewText  SentimentScore SentimentCategory  \
0  Average experience, nothing special.         -0.3089    Mixed Negative   
1            The quality is  top-notch.          0.0000          Positive   
2    Five stars for the quick delivery.          0.0000          Positive   
3   Good quality, but could be cheaper.          0.2382    Mixed Positive   
4  Average experience, nothing special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  


In [6]:
customer_reviews_df

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket
0,1,77,18,23-12-2023,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0
1,2,80,19,25-12-2024,5,The quality is top-notch.,0.0000,Positive,0.0 to 0.49
2,3,50,13,26-01-2025,4,Five stars for the quick delivery.,0.0000,Positive,0.0 to 0.49
3,4,78,15,21-04-2025,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive,0.0 to 0.49
4,5,64,2,16-07-2023,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0
...,...,...,...,...,...,...,...,...,...
1358,1359,28,4,25-05-2023,3,Not worth the money.,-0.1695,Mixed Negative,-0.49 to 0.0
1359,1360,58,12,13-11-2023,2,"Average experience, nothing special.",-0.3089,Negative,-0.49 to 0.0
1360,1361,96,15,07-03-2023,5,Customer support was very helpful.,0.6997,Positive,0.5 to 1.0
1361,1362,99,2,03-12-2025,1,Product did not meet my expectations.,0.0000,Negative,0.0 to 0.49


In [7]:
# Save the final DataFrame to a CSV file
customer_reviews_df.to_csv('fact_customer_reviews_with_sentiment.csv', index=False)

In [8]:
from google.colab import files
files.download('fact_customer_reviews_with_sentiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>